*** Code has been heavily influence from pix2struct finetuning notebook: https://colab.research.google.com/drive/1lbqiSiA0sDF7JDWPeS0tccrM85LloVha?usp=sharing ***

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets

# Load Dataset From Hugging Face

In [6]:
from datasets import load_dataset

In [7]:
all_train= load_dataset('hk-kaden-kim/pix2struct-chartcaptioning')
train_dataset = all_train['train']
val_dataset = all_train['validation']

# Load Model From Hugging Face

In [1]:
from transformers import AutoProcessor, Pix2StructForConditionalGeneration

processor = AutoProcessor.from_pretrained("google/pix2struct-base", is_vqa=False)
model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-base", is_vqa=False)

# Create PyTorch Dataset

In [2]:
from torch.utils.data import Dataset, DataLoader

MAX_PATCHES = 2028

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], return_tensors="pt", add_special_tokens=True, max_patches=MAX_PATCHES)

        encoding = {k:v.squeeze() for k,v in encoding.items()}
        encoding["caption"] = item["caption"]
        return encoding


In [3]:

def collator(batch):
    new_batch = {"flattened_patches": [], "attention_mask": []}
    texts = [item["caption"] for item in batch]
    text_inputs = processor(text=texts, truncation=False, padding="longest", return_tensors="pt", add_special_tokens=True)
    new_batch["labels"] = text_inputs.input_ids
    for item in batch:
        new_batch["flattened_patches"].append(item["flattened_patches"])
        new_batch["attention_mask"].append(item["attention_mask"])
    new_batch["flattened_patches"] = torch.stack(new_batch["flattened_patches"])
    new_batch["attention_mask"] = torch.stack(new_batch["attention_mask"])
    return new_batch

# Train the model

In [8]:
train_dataset_icd = ImageCaptioningDataset(train_dataset, processor)
train_dataloader = DataLoader(train_dataset_icd, shuffle=True, batch_size=2, collate_fn=collator)
val_dataset_icd = ImageCaptioningDataset(val_dataset, processor)
val_dataloader = DataLoader(val_dataset_icd, shuffle=False, batch_size=2, collate_fn=collator)

In [9]:
# Define EarlyStopping class for monitoring training
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path="checkpoint_name.pt"):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

    def trace_func(self, message):
        print(message)




In [ ]:
import torch
import numpy as np

# Function to evaluate the model on the validation set
def evaluate(model, val_dataloader, device):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            labels = batch["labels"].to(device)
            flattened_patches = batch["flattened_patches"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model(flattened_patches=flattened_patches, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.mean().item()
    model.train()
    return val_loss / len(val_dataloader)

EPOCHS = 5000
# Training setup
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.95)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.nn.DataParallel(model)
model.to(device)
model.train()

# Training loop
print("Start Training")
early_stopping = EarlyStopping(patience=10, verbose=True)
for epoch in range(EPOCHS):
    print(f"Epoch: {epoch}")
    for idx, batch in enumerate(train_dataloader):
        labels = batch.pop("labels").to(device)
        flattened_patches = batch.pop("flattened_patches").to(device)
        attention_mask = batch.pop("attention_mask").to(device)
        outputs = model(flattened_patches=flattened_patches, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss = loss.mean()
        print(f"Loss: {loss.item()}")
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    val_loss = evaluate(model, val_dataloader, device)
    # val_loss = val_loss.mean()
    print(f'Validation Loss: {val_loss:.6f}')
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    if (epoch + 1) % 1000 == 0:
        model.eval()
        predictions = model.generate(flattened_patches=flattened_patches, attention_mask=attention_mask)
        print(f"Predictions: {processor.batch_decode(predictions, skip_special_tokens=True)}")
        model.train()


# Specify the path where you want to save your model
save_directory = "./save_path"

# Save model and tokenizer using the `save_pretrained` method
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(save_directory)
processor.save_pretrained(save_directory)

# Save the PyTorch model weights in .pth format
torch.save(model_to_save.state_dict(), "name.pth")